In [6]:
from snorkel.labeling import LabelingFunction, PandasLFApplier
import nltk 

from snorkel.labeling import labeling_function 

from snorkel.labeling import PandasLFApplier 

from snorkel.labeling import LFAnalysis 

from nltk.corpus import movie_reviews 

import nltk 

import random 

import pandas as pd 


from snorkel.labeling import LabelingFunction
import numpy as np
import pandas as pd

# Define the constants
ABSTAIN = -1
POS = 0
NEG = 1

# Create a DataFrame with more data
df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6, 7, 8],
    'review': [
        "This movie was absolutely wonderful!",
        "The film was terrible and boring.",
        "I have mixed feelings about the movie.",
        "I have no opinion about the movie.",
        "The movie was fantastic and exciting!",
        "I didn't like the movie, it was too slow.",
        "The movie was okay, not great but not bad either.",
        "The movie was confusing and dull."
    ]
})

# Split the DataFrame into a training set and a test set
df_train = df.iloc[:6]  # First 6 records for training
df_test = df.iloc[6:]  # Remaining records for testing

# Define the true labels for the test set
Y_test = [ABSTAIN, NEG]  # Replace this with the actual labels

# Convert Y_test to a NumPy array
Y_test = np.array(Y_test)

@labeling_function()
def lf_positive_review(x):
    return POS if 'wonderful' in x.review or 'fantastic' in x.review else ABSTAIN 

@labeling_function()
def lf_negative_review(x):
    return NEG if 'terrible' in x.review or 'boring' in x.review or 'slow' in x.review or 'dull' in x.review else ABSTAIN 

@labeling_function()
def lf_neutral_review(x):
    return ABSTAIN if 'mixed feelings' in x.review or 'no opinion' in x.review or 'okay' in x.review else ABSTAIN 

# Apply the labeling functions to the training set and the test set
lfs = [lf_positive_review, lf_negative_review, lf_neutral_review]
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

print(L_train)
print(L_test)
print(L_test.shape)
print(Y_test.shape)

100%|██████████| 2/2 [00:00<00:00, 3269.14it/s]

[[ 0 -1 -1]
 [-1  1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [ 0 -1 -1]
 [-1  1 -1]]
[[-1 -1 -1]
 [-1  1 -1]]
(2, 3)
(2,)


In [7]:
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
majority_model = MajorityLabelVoter()
majority_model.predict(L=L_train)
majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")["accuracy"]
print( majority_acc)

1.0


In [8]:
preds_train = majority_model.predict(L=L_train)
print(preds_train)

[ 0  1 -1 -1  0  1]


In [9]:
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)
label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(label_model_acc)

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.058]
INFO:root:[100 epochs]: TRAIN:[loss=0.000]
INFO:root:[200 epochs]: TRAIN:[loss=0.000]
 46%|████▌     | 231/500 [00:00<00:00, 2306.08epoch/s]INFO:root:[300 epochs]: TRAIN:[loss=0.000]
INFO:root:[400 epochs]: TRAIN:[loss=0.000]
100%|██████████| 500/500 [00:00<00:00, 2896.35epoch/s]
INFO:root:Finished Training


1.0


In [10]:
# Predict the labels for the training data
Y_train_pred = label_model.predict(L=L_train)

# Print the predicted labels
print(Y_train_pred)

[ 0  1 -1 -1  0  1]


In [11]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lf_positive_review,0,[0],0.333333,0.0,0.0
lf_negative_review,1,[1],0.333333,0.0,0.0
lf_neutral_review,2,[],0.000000,0.0,0.0


In [12]:
# Create a DataFrame with the predicted labels
df_train_pred = df_train.copy()
df_train_pred['predicted_label'] = Y_train_pred

# Display the DataFrame
print(df_train_pred)

   id                                     review  predicted_label
0   1       This movie was absolutely wonderful!                0
1   2          The film was terrible and boring.                1
2   3     I have mixed feelings about the movie.               -1
3   4         I have no opinion about the movie.               -1
4   5      The movie was fantastic and exciting!                0
5   6  I didn't like the movie, it was too slow.                1


----end---
not required below cells

In [14]:
probs_train = label_model.predict_proba(L=L_train)
print(probs_train)

[[0.98522167 0.01477833]
 [0.01477833 0.98522167]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.98522167 0.01477833]
 [0.01477833 0.98522167]]


In [15]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=probs_train, L=L_train
)

In [17]:
print(df_train_filtered)
print(probs_train_filtered)

   id                                     review
0   1       This movie was absolutely wonderful!
1   2          The film was terrible and boring.
4   5      The movie was fantastic and exciting!
5   6  I didn't like the movie, it was too slow.
[[0.98522167 0.01477833]
 [0.01477833 0.98522167]
 [0.98522167 0.01477833]
 [0.01477833 0.98522167]]


In [29]:
from snorkel.utils import probs_to_preds
X_train = df_train_filtered['review'].tolist()
X_test = df_test['review'].tolist()
preds_train_filtered = probs_to_preds(probs=probs_train_filtered)
print(preds_train_filtered)
print(X_train)

[0 1 0 1]
['This movie was absolutely wonderful!', 'The film was terrible and boring.', 'The movie was fantastic and exciting!', "I didn't like the movie, it was too slow."]
